In [47]:
import pandas as py
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import cascaded_union
import folium
from shapely.geometry import Polygon
from folium.features import GeoJson
import streamlit as st
from streamlit_folium import folium_static

In [2]:
A1_traffic_accident_simple = gpd.read_file("/Users/guoyanting/Desktop/NTUDAC/專案/Gogoro/rawData/traffic_accident/traffic_accident_A1/A1_simple/A1_simple.shp")

In [3]:
A2_traffic_accident_simple = gpd.read_file("/Users/guoyanting/Desktop/NTUDAC/專案/Gogoro/rawData/traffic_accident/traffic_accident_A2/A2_simple/A2_simple.shp")

## Model

In [4]:
polygon_25 = Polygon([(121.550056, 25.045366), (121.540898, 25.047731), (121.532513, 25.042877), 
                         (121.526948, 25.033459), (121.533632, 25.020575), (121.539758, 25.017153), 
                         (121.552458, 25.020729), (121.559822, 25.028321), (121.559395, 25.038167), 
                         (121.550056, 25.045366)])

polygon_50 = Polygon([(121.554402, 25.033872), (121.542710, 25.030000), (121.534513, 25.035225), 
                      (121.540201, 25.037896), (121.554402, 25.033872)])

polygon_75 = Polygon([(121.544402, 25.033872), (121.542710, 25.033694), (121.541880, 25.035225), 
                      (121.543201, 25.035896), (121.544402, 25.033872)])

In [10]:
# 創建一個 folium 地圖
m = folium.Map(location=[25.044281, 121.517532], zoom_start=13)

# 創建三個 GeoJson 物件，分別代表三個 polygon
polygon_25_geojson = GeoJson(polygon_25.__geo_interface__, style_function=lambda x: {'fillColor': 'green', 'color': 'green'})
polygon_50_geojson = GeoJson(polygon_50.__geo_interface__, style_function=lambda x: {'fillColor': 'red', 'color': 'red'})
polygon_75_geojson = GeoJson(polygon_75.__geo_interface__, style_function=lambda x: {'fillColor': 'blue', 'color': 'blue'})

# 將三個 GeoJson 物件加入地圖中
polygon_25_geojson.add_to(m)
polygon_50_geojson.add_to(m)
polygon_75_geojson.add_to(m)

# 顯示地圖
m

In [5]:
# 建立 GeoDataFrame
polygon_appear = gpd.GeoDataFrame(geometry=[polygon_25, polygon_50, polygon_75], crs='EPSG:4326')

In [22]:
total_risk = 0

for index in reversed(polygon_appear.index):
    print(index)

    A1_points_within_polygon = A1_traffic_accident_simple[A1_traffic_accident_simple.geometry.within(polygon_appear.iloc[index].geometry)]
    print(A1_points_within_polygon)
    A1_risk_sum = (A1_points_within_polygon['danger_val'].sum())*0.25

    A2_points_within_polygon = A2_traffic_accident_simple[A2_traffic_accident_simple.geometry.within(polygon_appear.iloc[index].geometry)]
    print(A2_points_within_polygon)
    A2_risk_sum = (A2_points_within_polygon['danger_val'].sum())*0.25

    total_risk += A1_risk_sum + A2_risk_sum


2
Empty GeoDataFrame
Columns: [danger_val, geometry]
Index: []
         danger_val                    geometry
29691             1  POINT (121.54326 25.03414)
112852            1  POINT (121.54353 25.03434)
422778            1  POINT (121.54373 25.03438)
422779            1  POINT (121.54373 25.03438)
453906            1  POINT (121.54356 25.03432)
489747            1  POINT (121.54300 25.03434)
544342            1  POINT (121.54357 25.03447)
544343            1  POINT (121.54357 25.03447)
554031            2  POINT (121.54302 25.03505)
554032            2  POINT (121.54302 25.03505)
581689            1  POINT (121.54411 25.03404)
581690            1  POINT (121.54411 25.03404)
638989            1  POINT (121.54349 25.03408)
675995            2  POINT (121.54330 25.03498)
766996            1  POINT (121.54351 25.03440)
843766            1  POINT (121.54299 25.03435)
866364            1  POINT (121.54340 25.03390)
876561            1  POINT (121.54357 25.03408)
876562            1  POIN

In [23]:
total_risk

4553.25

In [43]:
A1_points_within_polygon.describe()

,danger_val
count,35.000000
mean,10.457143
std,0.657216
min,10.000000
25%,10.000000
50%,10.000000
75%,11.000000
max,12.000000


In [42]:
# 創建一個 folium 地圖
m = folium.Map(location=[25.044281, 121.517532], zoom_start=13)

# 創建三個 GeoJson 物件，分別代表三個 polygon
polygon_25_geojson = GeoJson(polygon_25.__geo_interface__, style_function=lambda x: {'fillColor': 'green', 'color': 'green'})
polygon_50_geojson = GeoJson(polygon_50.__geo_interface__, style_function=lambda x: {'fillColor': 'red', 'color': 'red'})
polygon_75_geojson = GeoJson(polygon_75.__geo_interface__, style_function=lambda x: {'fillColor': 'blue', 'color': 'blue'})

A1_geojson = GeoJson(A1_points_within_polygon)

# 將四個 GeoJson 物件加入地圖中
polygon_25_geojson.add_to(m)
polygon_50_geojson.add_to(m)
polygon_75_geojson.add_to(m)
A1_geojson.add_to(m)

# 顯示地圖
m